In [1]:
import re
from urllib.request import urlopen  # b_soup_1.py
from bs4 import BeautifulSoup
import pandas as pd
from urllib.error import URLError
import pandas as pd
import requests
import socket
from urllib.error import URLError, HTTPError
from urllib.request import Request, urlopen
import os 
import pickle
from newspaper import Article
import numpy as np
from flair.models import SequenceTagger
from flair.data import Sentence
tagger = SequenceTagger.load('ner-ontonotes-large')

2022-06-09 15:07:07,273 loading file /Users/yitingzhou/.flair/models/ner-english-ontonotes-large/2da6c2cdd76e59113033adf670340bfd820f0301ae2e39204d67ba2dc276cc28.ec1bdb304b6c66111532c3b1fc6e522460ae73f1901848a4d0362cdf9760edb1
2022-06-09 15:07:34,255 SequenceTagger predicts: Dictionary with 76 tags: <unk>, O, B-CARDINAL, E-CARDINAL, S-PERSON, S-CARDINAL, S-PRODUCT, B-PRODUCT, I-PRODUCT, E-PRODUCT, B-WORK_OF_ART, I-WORK_OF_ART, E-WORK_OF_ART, B-PERSON, E-PERSON, S-GPE, B-DATE, I-DATE, E-DATE, S-ORDINAL, S-LANGUAGE, I-PERSON, S-EVENT, S-DATE, B-QUANTITY, E-QUANTITY, S-TIME, B-TIME, I-TIME, E-TIME, B-GPE, E-GPE, S-ORG, I-GPE, S-NORP, B-FAC, I-FAC, E-FAC, B-NORP, E-NORP, S-PERCENT, B-ORG, E-ORG, B-LANGUAGE, E-LANGUAGE, I-CARDINAL, I-ORG, S-WORK_OF_ART, I-QUANTITY, B-MONEY


In [2]:
## readin all dataset
def read_all_data(name):
    all_Data = pd.read_csv(name).drop_duplicates(subset=['source_link'])
    all_Data.index.name = 'id'
    all_Data = all_Data[all_Data['source_link'].notnull()]
    return all_Data

In [3]:
def get_news_text(df):
    dates = []
    text = []
    titles = []
    idx_lst = []
    keywords = []
    summary = []
    links = []
    for i, (idx, row) in enumerate(df.iterrows()):
        article = Article(row['source_link'])
        try:
            article.download()
            article.parse()
            article.nlp()
            dates.append(article.publish_date)
            text.append(article.text)
            titles.append(article.title)
            idx_lst.append(idx)
            keywords.append(article.keywords)
            summary.append(article.summary)
            links.append(row['source_link'])
        except:
            continue
    content = {'id': idx_lst, 'links': links, 'dates': dates, 'titles': titles, 'text': text, 'keywords': keywords, 'summary': summary}  
    df = pd.DataFrame(content)
    return df

In [4]:
def filter_by_model(model, df):
    loaded_model = pickle.load(open(model, 'rb'))
    filter_text = pd.read_csv(df)
    filter_text = filter_text[filter_text['titles'].notnull()]
    pred_res = loaded_model.predict(list(filter_text['titles']))
    filter_text['pred_res'] = pred_res
    filter_text = filter_text[filter_text['pred_res'] == 'pos']
    return filter_text

In [5]:
def get_nondate(df):
    non_dates = []
    for idx, row in df.iterrows():
        if type(df['dates']) == float:
            non_dates.append(row['id'])
    return non_dates

In [6]:
def update_date(df, nond_dates):
    datepublish = []
    date_dict = {}
    for idx, i in enumerate(non_dates):
        try:
            req = Request(list(df[df['id'] == i]['links'])[0],headers={'User-Agent': 'Mozilla/5.0'})
            webpage = urlopen(req, timeout=10).read()
            if 'datepublish' in str(webpage).lower():
                text = str(webpage)[str(webpage).lower().index('datepublish'):][:300]
                sentence = Sentence(text)
                tagger.predict(sentence)
                for entity in sentence.get_spans('ner'):
                    if entity.tag == 'DATE':
                        date_dict[i] = entity.text
                        datepublish.append(i)
                        break
        except Exception:
            continue

    for idx, row in df.iterrows():
        if type(row['dates']) == float:
            if row['id'] in date_dict:
                df.loc[idx,'dates'] = date_dict[row['id']]
    return df

In [7]:
def get_trainset(df):
    GPE = []
    LOC = []
    DATE = []
    TIME = []
    INDEX = []
    SENTENCE = []
    for idx, row in df.iterrows():
        if type(row['text']) != float:
            for sent in nlp(row['text']).sents:
                sub_sent = sent.text.strip()
                if sub_sent:
                    sentence = Sentence(sub_sent)
                    tagger.predict(sentence)
                    sub_gpe = []
                    sub_date = []
                    sub_time = []
                    sub_loc = []
                    for entity in sentence.get_spans('ner'):
                        if entity.tag == 'GPE':
                            sub_gpe.append(entity.text)
                        if entity.tag == 'DATE':
                            sub_date.append(entity.text)
                        if entity.tag == 'TIME':
                            sub_time.append(entity.text)
                        if entity.tag == 'LOC':
                            sub_loc.append(entity.text)
                    if sub_loc or sub_date or sub_gpe or sub_time:
                        GPE.append('|'.join(sub_gpe))
                        LOC.append('|'.join(sub_loc))
                        DATE.append('|'.join(sub_date))
                        TIME.append('|'.join(sub_time))
                        INDEX.append(row['id'])
                        SENTENCE.append(sub_sent)
    res = pd.DataFrame({'id':INDEX, 'text': SENTENCE, 'GPE': GPE, 'LOC':LOC, 'DATE': DATE, 'TIME': TIME})
    return res

In [ ]:
original_data = read_all_data('nasa_global_landslide_catalog_point.csv')
filter_by_newspaper_date = get_news_text(original_data)
filter_by_model_data = filter_by_model('article_detection.model', filter_by_newspaper_date)
non_date_dict = get_nondate(filter_by_model_data)
final_filter_date = update_date(filter_by_model_data, non_date_dict)
training_date = get_trainset(final_filter_date)